In [1]:
# use decomon to check for the robutness

In [2]:
import keras
import numpy as np

# Load the MNIST data and split it into training and testing sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale the images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Reshape images to have an additional channel dimension (1, 28, 28)
x_train = np.expand_dims(x_train, 1)
x_test = np.expand_dims(x_test, 1)

# Convert class labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


/Users/ducoffe_m/Library/Python/3.9/lib/python/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/Users/ducoffe_m/Library/Python/3.9/lib/python/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [19]:
from keras import layers, Sequential

# Define the model architecture
model = Sequential(
    [
        layers.Input(shape=(1, 28, 28)),
        layers.Conv2D(32, kernel_size=(3, 3)),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3)),
        layers.Activation(activation="relu"),
        layers.Flatten(),
        layers.Dense(10)
    ]
)
model.summary()

train_model = Sequential(model.layers+[layers.Activation('softmax')])
train_model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
                    optimizer="adam", 
                    metrics=["accuracy"])
train_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1)




Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 32, 26, 26)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 32, 26, 26)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 24, 24)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 64, 24, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │       368,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 387,466 (1.48 MB)

 Trainable params: 387,466 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 16s 38ms/step - accuracy: 0.8001 - loss: 1.6820 - val_accuracy: 0.9745 - val_loss: 1.4878
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 14s 34ms/step - accuracy: 0.9716 - loss: 1.4911 - val_accuracy: 0.9830 - val_loss: 1.4783


In [4]:
from jacobinet.attacks import get_adv_model

In [54]:
pgd_model = get_adv_model(model, loss= 'logits', epsilon=1e-2, attack='pgd', n_iter=20) # add clipping input domain + FGSMModel with set_epsilon


In [55]:
x = layers.Input(shape=(1, 28, 28))
y = layers.Input((10,))

In [56]:
model_adv = keras.models.Model([x, y], [model(x), model(pgd_model([x,y]))]) # concatenate model and its adverarial prediction

In [57]:
model_adv.compile('adam', loss=[keras.losses.CategoricalCrossentropy(from_logits=True), keras.losses.CategoricalCrossentropy(from_logits=True)],
               metrics=['accuracy', 'accuracy'], loss_weights=[1, 0.5])

In [58]:
model_adv.fit([x_train, y_train], [y_train, y_train], batch_size=128, epochs=2, validation_split=0.1)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 225s 533ms/step - loss: 1.4527 - sequential_3_accuracy: 0.9762 - sequential_3_accuracy_1: 0.6271 - sequential_3_loss: 2.7141 - val_loss: 0.2660 - val_sequential_3_accuracy: 0.9863 - val_sequential_3_accuracy_1: 0.8777 - val_sequential_3_loss: 0.4307
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 806s 2s/step - loss: 0.2864 - sequential_3_accuracy: 0.9840 - sequential_3_accuracy_1: 0.8671 - sequential_3_loss: 0.4633 - val_loss: 0.2076 - val_sequential_3_accuracy: 0.9888 - val_sequential_3_accuracy_1: 0.9092 - val_sequential_3_loss: 0.3277


In [59]:
model_adv.compile('adam', loss=[keras.losses.CategoricalCrossentropy(from_logits=True), keras.losses.CategoricalCrossentropy(from_logits=True)],
               metrics=['accuracy', 'accuracy'], loss_weights=[1, 1])

In [60]:
model_adv.fit([x_train, y_train], [y_train, y_train], batch_size=128, epochs=4, validation_split=0.1)

Epoch 1/4
305/422 ━━━━━━━━━━━━━━━━━━━━ 1:00 515ms/step - loss: 0.3883 - sequential_3_accuracy: 0.9863 - sequential_3_accuracy_1: 0.8984 - sequential_3_loss: 0.3420

KeyboardInterrupt: 

In [61]:
model_adv.evaluate([x_test, y_test], [y_test, y_test])

313/313 ━━━━━━━━━━━━━━━━━━━━ 56s 179ms/step - loss: 0.3734 - sequential_3_accuracy: 0.9825 - sequential_3_accuracy_1: 0.9068 - sequential_3_loss: 0.3192


[0.315471351146698,
 0.2704598009586334,
 0.2704598009586334,
 0.9853000044822693,
 0.9215999841690063]

In [62]:
#fgsm_attack = torchattacks.FGSM(torch_model, eps=0.1)
#pgd_attack = torchattacks.PGD(torch_model, eps=0.1, steps=1000)
auto_attack = torchattacks.attacks.autoattack.AutoAttack(model, eps=0.2)

#.PGD(torch_lip_model, eps=10., steps=steps)

In [63]:
torch.tensor(y_train[:10].argmax(-1)).dtype

torch.int64

In [64]:
adv_data = auto_attack(torch.Tensor(x_train[:10]), torch.tensor(y_train[:10].argmax(-1)))

In [65]:
model.compile('adam', loss=keras.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [66]:
model.evaluate(adv_data, y_train[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9000 - loss: 0.2550


[0.2549683153629303, 0.8999999761581421]

In [67]:
adv_data_ = pgd_model([x_train[:10], y_train[:10]])

In [68]:
print(y_train[:10].argmax(-1), model(adv_data_).argmax(-1), model(adv_data).argmax(-1))

[5 0 4 1 9 2 1 3 1 4] tensor([3, 0, 4, 1, 9, 2, 1, 3, 1, 4], device='mps:0') tensor([3, 0, 4, 1, 9, 2, 1, 3, 1, 4], device='mps:0')
